In [4]:
from torch import nn
import wandb
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Normalization

In [5]:
#load data

path = '../dataset'
x_train = np.load(f'{path}/x_train.npy')
y_train = np.load(f'{path}/y_train.npy')
x_test = np.load(f'{path}/x_test.npy')
y_test = np.load(f'{path}/y_test.npy')


## Hyperparameter Tuning

In [9]:
learning_rate = 0.001
batch_size = 32
num_epochs = 5

## Model Training

### Model Definitions

In [10]:


# simple neural network

class SimpleNN(nn.Module):
	def __init__(self, input_size, num_classes):
		super(SimpleNN, self).__init__()
		self.fc1 = nn.Linear(input_size, int(input_size/2))
		self.relu = nn.ReLU()
		self.fc2 = nn.Linear(int(input_size/2), int(input_size/4))
		self.fc3 = nn.Linear(int(input_size/4), num_classes)

	def forward(self, x):
		out = self.fc1(x)
		out = self.relu(out)
		out = self.fc2(out)
		out = self.relu(out)
		out = self.fc3(out)
		return out


In [11]:
# convolutional neural network
# variable input length

class ConvNN(nn.Module):

	def __init__(self, num_classes):
		super(ConvNN, self).__init__()
		self.layer1 = nn.Sequential(
			nn.Conv1d(1, 16, kernel_size=5, stride=1, padding=2),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2))
		self.layer2 = nn.Sequential(
			nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2))
		self.fc = nn.Linear(7*7*32, num_classes)

In [12]:
# Make data loaders

from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


### Training Loop

In [21]:
# model training
def train_model(model, criterion, optimizer, num_epochs, train_loader, test_loader, wandb):
	total_step = len(train_loader)
	for epoch in range(num_epochs):
		for i, (images, labels) in enumerate(train_loader):
			images = images.to(device)
			labels = labels.to(device)
			outputs = model(images)
			print(labels.size(), outputs.size())
			loss = criterion(outputs, labels)
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			if (i+1) % 10 == 0:
				print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
					   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
				wandb.log({'training_loss': loss.item()})
		val_loss = 0
		with torch.no_grad():
			for i, (images, labels) in enumerate(test_loader):
				images = images.to(device)
				labels = labels.to(device)
				outputs = model(images)
				val_loss += criterion(outputs, labels)
		print('Validation loss: {:.4f}'.format(val_loss))
		wandb.log({'validation_loss': val_loss})
	# break
	return model

In [22]:
input_size = x_train.shape[1]
num_classes = y_train.shape[1]
lr = learning_rate
training_size = len(train_loader)
testing_size = len(test_loader)
model = SimpleNN(input_size, num_classes).to(device)
batch_size = batch_size
num_epochs = num_epochs
architecture = 'FNN' if isinstance(model, SimpleNN) else 'CNN'
print(architecture)

wandb.init(
    # set the wandb project where this run will be logged
    project="ml4Oxi",
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": architecture,
    "dataset": "MP_only",
    "epochs": num_epochs,
    }
)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model = train_model(model, criterion, optimizer, 5, train_loader, test_loader, wandb)


FNN
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
Epoch [1/5], Step [10/147], Loss: 0.0703
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
Epoch [1/5], Step [20/147], Loss: 0.0645
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8]) torch.Size([32

## Performance Analysis